In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-31 15:14:55,210.210 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-31 15:14:55,211.211 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [7]:
%pip install alpaca-py

     |████████████████████████████████| 95 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 299 kB 43.6 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 73.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 90.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.requests import StockBarsRequest
import pyarrow as pa
import pyarrow.parquet as pq

isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

In [9]:


ALPACA_API_KEY_ID = 'AK8MWGDK1GMSP91W1SGM'
ALPACA_API_SECRET_KEY = 'oM5h4WWGfuKrctHWdCmtpwLcOuoV5TLemJ5ImavI'


In [10]:
if False:
    fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
    bitcoin_etf_symbols = ['BITO']
    commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
    fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
    equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
    'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                          'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                          'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
    sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
    bench_symbol=['SPY','QQQ','IVV']
    all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
    #all_symbols
    columns_to_capture = ['close','open','symbol']
    data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
    parquet_file_out =f"{data_folder}/etf_market_data.parquet"

In [11]:
#import boto3
#dynamodb = boto3.resource('dynamodb')
#financial_indexes = dynamodb.Table('financial_indexes')
#financial_indexes.get_item(Key={"symbol":{"S":"SPY"}})
#syms=pd.DataFrame([{'symbol':'SPY'}])
#ret_df= get_new_data_df(syms)
#ret_df

In [ ]:
def get_new_data_df_alpaca(symbol,start_date="2019-01-02",end_date="2023-01-30"):
    if type(symbol)==list:
        symbols=symbol
    else:
        local_syms=str(symbol.values[0][0])
        symbols=[local_syms]
    
    ALPACA_API_KEY_ID = 'AK8MWGDK1GMSP91W1SGM'
    ALPACA_API_SECRET_KEY = 'oM5h4WWGfuKrctHWdCmtpwLcOuoV5TLemJ5ImavI'
    data_client = StockHistoricalDataClient(ALPACA_API_KEY_ID, ALPACA_API_SECRET_KEY)

    # Set parameters

    start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
    end_date = pd.to_datetime(end_date).tz_localize('America/New_York')

    timeframe_30_min = TimeFrame(30, TimeFrameUnit.Minute)

    request_parameters = StockBarsRequest(
                    symbol_or_symbols=symbols,
                    timeframe=timeframe_30_min,
                    start=start_date,
                    end=end_date,
                    )

    # Fetch data and convert to dataframe
    min_30_bars = data_client.get_stock_bars(request_parameters).df.reset_index()
    min_30_bars['date']=min_30_bars.timestamp.dt.date
    min_30_bars['minute'] = (min_30_bars.timestamp.dt.hour.astype(str).str.pad(2,fillchar='0'))+":"+(min_30_bars.timestamp.dt.minute.astype(str).str.pad(2,fillchar='0'))
    return(min_30_bars)
sym_list=['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU','SPY',
          'QQQ','VTI','VTV','BND','AGG','VUG','VIG','IJR','IJH','IEMG','IWF','IWD','GLD','IWM','VYM','VXUS','VO','EFA','SCHD','ARKK']
etf_pd = get_new_data_df_alpaca(sym_list)

#syms=pd.DataFrame([{'symbol':'XLC'}])
#etf_pd = get_new_data_df_alpaca(syms)

In [ ]:
etf_df= spark.createDataFrame(etf_pd.reset_index())
etf_parquet_file = 's3://fsidatalake/intraday_market_data/etf_30mins.parquet'
etf_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(etf_parquet_file)

In [ ]:
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
#syms=pd.DataFrame([{'symbol':'SPY'}])
#get_new_data_df(syms)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        print(final_url)
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        ret_pd['symbol']=local_sym
        for one_col in ['open','high','low','close']:
            ret_pd[f'market{one_col.capitalize()}'] = np.where(ret_pd[f'market{one_col.capitalize()}'].isnull(),ret_pd[one_col],ret_pd[f'market{one_col.capitalize()}'] )
            ret_pd[one_col] = np.where(ret_pd[one_col].isnull(),ret_pd[f'market{one_col.capitalize()}'],ret_pd[one_col] )
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'minute':None,
                              'open':None,
                              'high':None,
                              'low':None,
                              'close':None,
                              'notional':None,
                              'numberOfTrades':None,
                              'symbol': local_sym,
                              'marketOpen':None,
                              'marketHigh':None,
                              'marketLow':None,
                              'marketClose':None,
                              'marketNotional':None,
                              'marketNumberOfTrades':None
                             }])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
  

In [ ]:
refresh_data =False
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.sp500 = [ "AAPL","MSFT",
  "AMZN",
  "GOOGL",
  "BRK.B",
  "GOOG",
  "NVDA",
  "TSLA",
  "XOM",
  "UNH",
  "JNJ",
  "JPM",
  "V",
  "META",
  "PG",
  "HD",
  "CVX",
  "MA",
  "LLY",
  "MRK",
  "ABBV",
  "BAC",
  "PFE",
  "AVGO",
  "KO",
  "PEP",
  "TMO",
  "COST",
  "WMT",
  "DIS",
  "MCD",
  "CSCO",
  "ABT",
  "WFC",
  "ACN",
  "DHR",
  "ADBE",
  "CMCSA",
  "VZ",
  "CRM",
  "PM",
  "NKE",
  "NFLX",
  "LIN",
  "TXN",
  "COP",
  "BMY",
  "NEE",
  "QCOM",
  "RTX",
  "T",
  "HON",
  "CAT",
  "ORCL",
  "AMGN",
  "UPS",
  "MS",
  "LOW",
  "SBUX",
  "UNP",
  "SPGI",
  "IBM",
  "AMD",
  "GS",
  "PLD",
  "INTU",
  "BA",
  "ELV",
  "INTC",
  "CVS",
  "DE",
  "BLK",
  "SCHW",
  "MDT",
  "LMT",
  "GILD",
  "AXP",
  "AMT",
  "C",
  "AMAT",
  "BKNG",
  "TJX",
  "CB",
  "CI",
  "PYPL",
  "NOW",
  "ADP",
  "GE",
  "ADI",
  "MDLZ",
  "TMUS",
  "ISRG",
  "MMC",
  "SYK",
  "VRTX",
  "SLB",
  "REGN",
  "MO",
  "EOG",
  "PGR",
  "DUK",
  "TGT",
  "ZTS",
  "SO",
  "BDX",
  "APD",
  "MU",
  "FISV",
  "EQIX",
  "AON",
  "LRCX",
  "USB",
  "PNC",
  "BSX",
  "TFC",
  "ITW",
  "ETN",
  "FCX",
  "MMM",
  "NOC",
  "CCI",
  "CSX",
  "CME",
  "MRNA",
  "EL",
  "MPC",
  "HUM",
  "ICE",
  "CL",
  "WM",
  "KLAC",
  "PXD",
  "NSC",
  "VLO",
  "HCA",
  "ATVI",
  "SNPS",
  "MCK",
  "GM",
  "SHW",
  "DG",
  "EMR",
  "F",
  "PSX",
  "D",
  "GD",
  "SRE",
  "CDNS",
  "MCO",
  "OXY",
  "EW",
  "ORLY",
  "MET",
  "AEP",
  "NXPI",
  "JCI",
  "PSA",
  "AIG",
  "MAR",
  "APH",
  "A",
  "GIS",
  "ROP",
  "ADM",
  "CTVA",
  "FDX",
  "ADSK",
  "AZO",
  "COF",
  "FIS",
  "TRV",
  "CMG",
  "NUE",
  "KMB",
  "HES",
  "CNC",
  "O",
  "IQV",
  "MCHP",
  "DVN",
  "CHTR",
  "MSI",
  "DOW",
  "NEM",
  "BIIB",
  "MSCI",
  "AFL",
  "SPG",
  "DXCM",
  "ROST",
  "EXC",
  "TT",
  "PH",
  "AJG",
  "IDXX",
  "LHX",
  "TEL",
  "SYY",
  "HLT",
  "MNST",
  "PCAR",
  "PRU",
  "WMB",
  "CTAS",
  "XEL",
  "ECL",
  "STZ",
  "AMP",
  "KMI",
  "HAL",
  "DD",
  "BK",
  "CARR",
  "TDG",
  "YUM",
  "PAYX",
  "WELL",
  "CMI",
  "ALL",
  "FTNT",
  "MTD",
  "OTIS",
  "EA",
  "CTSH",
  "ED",
  "ILMN",
  "STT",
  "ALB",
  "RMD",
  "AME",
  "ROK",
  "VICI",
  "WBD",
  "HSY",
  "DFS",
  "DLR",
  "KEYS",
  "ON",
  "CSGP",
  "DLTR",
  "BKR",
  "KHC",
  "GPN",
  "SBAC",
  "ANET",
  "OKE",
  "ODFL",
  "DHI",
  "URI",
  "PEG",
  "APTV",
  "PPG",
  "KDP",
  "KR",
  "WEC",
  "CPRT",
  "AWK",
  "IFF",
  "FAST",
  "ENPH",
  "CEG",
  "VRSK",
  "ES",
  "GLW",
  "WTW",
  "MTB",
  "CBRE",
  "EBAY",
  "FANG",
  "EFX",
  "WBA",
  "ABC",
  "HPQ",
  "ZBH",
  "EIX",
  "ULTA",
  "IT",
  "TROW",
  "CDW",
  "PCG",
  "GWW",
  "FRC",
  "LEN",
  "GEHC",
  "WY",
  "RSG",
  "TSCO",
  "AVB",
  "FITB",
  "HIG",
  "DAL",
  "LYB",
  "VMC",
  "ARE",
  "FTV",
  "ACGL",
  "GPC",
  "BAX",
  "ANSS",
  "LH",
  "AEE",
  "FE",
  "IR",
  "ETR",
  "RF",
  "DTE",
  "PPL",
  "RJF",
  "LUV",
  "PFG",
  "HBAN",
  "MLM",
  "EQR",
  "CFG",
  "PWR",
  "EXR",
  "HPE",
  "HOLX",
  "DOV",
  "STE",
  "NDAQ",
  "VTR",
  "VRSN",
  "CTRA",
  "CAH",
  "NTRS",
  "WAT",
  "STLD",
  "WST",
  "ALGN",
  "EPAM",
  "TDY",
  "LVS",
  "CHD",
  "TSN",
  "MPWR",
  "INVH",
  "MAA",
  "WAB",
  "MKC",
  "CNP",
  "XYL",
  "DRI",
  "BALL",
  "MRO",
  "CMS",
  "AMCR",
  "IEX",
  "TTWO",
  "FSLR",
  "SWKS",
  "AES",
  "BR",
  "EXPD",
  "SIVB",
  "KEY",
  "MOH",
  "OMC",
  "PKI",
  "K",
  "EXPE",
  "CAG",
  "ETSY",
  "BBY",
  "CLX",
  "MOS",
  "TRGP",
  "DGX",
  "SEDG",
  "COO",
  "CINF",
  "SYF",
  "FMC",
  "CF",
  "ZBRA",
  "TER",
  "SJM",
  "ATO",
  "UAL",
  "INCY",
  "FDS",
  "JBHT",
  "IRM",
  "NVR",
  "J",
  "PAYC",
  "AVY",
  "FLT",
  "TXT",
  "GRMN",
  "MTCH",
  "POOL",
  "LKQ",
  "APA",
  "HWM",
  "PEAK",
  "NTAP",
  "TRMB",
  "ESS",
  "VTRS",
  "PTC",
  "LW",
  "WRB",
  "MKTX",
  "EVRG",
  "WDC",
  "RCL",
  "IPG",
  "KIM",
  "AKAM",
  "RE",
  "IP",
  "TYL",
  "LNT",
  "STX",
  "MGM",
  "BRO",
  "JKHY",
  "LDOS",
  "GEN",
  "HST",
  "SNA",
  "PKG",
  "HRL",
  "NDSN",
  "CPT",
  "CBOE",
  "UDR",
  "DPZ",
  "SWK",
  "TECH",
  "CRL",
  "PHM",
  "CHRW",
  "BF.B",
  "EQT",
  "CE",
  "HSIC",
  "L",
  "PARA",
  "QRVO",
  "MAS",
  "LYV",
  "TFX",
  "KMX",
  "CZR",
  "NI",
  "CDAY",
  "TPR",
  "BWA",
  "GL",
  "WYNN",
  "CCL",
  "EMN",
  "AAL",
  "FOXA",
  "BXP",
  "CPB",
  "JNPR",
  "BIO",
  "BBWI",
  "REG",
  "ALLE",
  "VFC",
  "UHS",
  "WRK",
  "TAP",
  "CTLT",
  "CMA",
  "RHI",
  "AAP",
  "FFIV",
  "HII",
  "PNR",
  "WHR",
  "BEN",
  "ROL",
  "PNW",
  "IVZ",
  "FRT",
  "ZION",
  "XRAY",
  "NWSA",
  "SEE",
  "SBNY",
  "NRG",
  "AOS",
  "OGN",
  "HAS",
  "GNRC",
  "AIZ",
  "DXC",
  "ALK",
  "NCLH",
  "MHK",
  "NWL",
  "LNC",
  "RL",
  "LUMN",
  "FOX",
  "DVA",
  "DISH",
  "NWS"
 ]
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors+\
                            self.sp500
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','minute','open','high','low','close','notional','numberOfTrades','symbol'
                                   ,'marketOpen','marketHigh','marketLow','marketClose','marketNotional','marketNumberOfTrades']
        self.type_excl_field_map = {'date':'timestamp','minute':'string','symbol':'string'}
        self.pandas_udf_schema =[f"{i} double" for i in self.columns_to_capture if (i not in self.type_excl_field_map.keys()) ] +\
                                    [f"{i[0]} {i[1]}" for i in self.type_excl_field_map.items()]
        
        self.data_folder = 's3://fsidatalake/intraday_market_data/'
        self.parquet_file_out =f"{self.data_folder}/OHLC_30m.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/1mm?token={}&chartIEXOnly=true'
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            #self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
            self.old_pd=self.all_symbols_df.toPandas()
            self.old_pd['date']=datetime.datetime.now()-datetime.timedelta(days=365*50)
            self.old_df= spark.createDataFrame(self.old_pd)
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema= ','.join(self.pandas_udf_schema))\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def set_sp500_constituents(self):
        pass
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
        

mdg=MarketDataGateway()
mdg.set_highwatermarks()
# set bc vars
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
iex_token_bc =sc.broadcast(mdg.iex_token)
iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
# set bc vars
if refresh_data:
    mdg.set_new_symbol_df()
    mdg.write_new_symbol_df()
    print(mdg.write_stats.toPandas().sort_values(['symbol']))
#mdg.rewrite_if_dups()

In [ ]:

old_df_after_write = spark.read.parquet(mdg.parquet_file_out)
dup_df = old_df_after_write.groupby('date','minute','symbol').count()
dup_count=dup_df.where('count>1').count()
dup_count
dup_df.toPandas().query("symbol=='AMZN'")

In [ ]:
if False:
    sym_sample=spark.read.parquet(mdg.parquet_file_out)
else:
    sym_sample=spark.read.parquet(etf_parquet_file)
sym_sample.count()

In [ ]:
sym_sample_pd = sym_sample.toPandas()

In [ ]:
sym_sample_pd['open_close']=((sym_sample_pd['close']/sym_sample_pd['open']-1))*100#+(sym_sample_pd['marketClose']/sym_sample_pd['marketOpen']-1))/2
sym_sample_first_last=sym_sample_pd.query('minute=="09:30" or minute=="15:30"')
sym_pd_pivot=pd.pivot_table(sym_sample_first_last,values=['open_close','open','close'],index=['symbol','date'],columns=['minute'],aggfunc='mean').dropna()
sym_pd_pivot.columns = sym_pd_pivot.columns.to_series().str.join('_')
sym_pd_pivot['day_change']=((sym_pd_pivot['close_15:30']/sym_pd_pivot['close_09:30']-1))*100
sym_pd_pivot['open_close_09:30_SIGN'] = [1 if x > 0 else (0 if x==0 else -1) for x in sym_pd_pivot['open_close_09:30']]
sym_pd_pivot['open_close_15:30_SIGN'] = [1 if x > 0 else (0 if x==0 else -1) for x in sym_pd_pivot['open_close_15:30']]
sym_pd_pivot['day_change_SIGN'] = [1 if x > 0 else (0 if x==0 else -1) for x in sym_pd_pivot['day_change']]
sym_pd_pivot['open_close_pnl']= ((sym_pd_pivot['open_close_09:30_SIGN']==sym_pd_pivot['open_close_15:30_SIGN'])&sym_pd_pivot['open_close_09:30_SIGN']!=0)*sym_pd_pivot['open_close_15:30']
sym_pd_pivot['day_change_pnl']= ((sym_pd_pivot['open_close_09:30_SIGN']==sym_pd_pivot['day_change_SIGN'])&sym_pd_pivot['open_close_09:30_SIGN']!=0)*sym_pd_pivot['day_change']

In [ ]:
sym_pd_pivot.groupby('symbol').agg({'open_close_pnl':['count','mean','sum'],'day_change':['mean','sum']}).style

close_09:30                                  c_l_o_s_e___0_9_:_3_0
close_15:30                                  c_l_o_s_e___1_5_:_3_0
open_09:30                                     o_p_e_n___0_9_:_3_0
open_15:30                                     o_p_e_n___1_5_:_3_0
open_close_09:30                   o_p_e_n___c_l_o_s_e___0_9_:_3_0
open_close_15:30                   o_p_e_n___c_l_o_s_e___1_5_:_3_0
day_change                                     d_a_y___c_h_a_n_g_e
open_close_09:30_SIGN    o_p_e_n___c_l_o_s_e___0_9_:_3_0___S_I_G_N
open_close_15:30_SIGN    o_p_e_n___c_l_o_s_e___1_5_:_3_0___S_I_G_N
day_change_SIGN                      d_a_y___c_h_a_n_g_e___S_I_G_N
open_close_pnl                         o_p_e_n___c_l_o_s_e___p_n_l
day_change_pnl                         d_a_y___c_h_a_n_g_e___p_n_l
dtype: object